In [1]:
import glob
import pandas as pd

listOfFileNames = []
listOfClassNames = []
listOfDataPerClass = []
files = []
classNames = []
#path = "/Users/user/Desktop/asd/thesis/placeholder/"
#path = "/Users/user/Desktop/pinakalatest/thesis/placeholder/"
path = "non-signer_data/2 arm gestures/"
classNames = glob.glob(path + "*")

for c in classNames:
    className = c.split("\\")[1]
    listOfClassNames.append(c.split("\\")[1])
    files = glob.glob(path + className+"/*") 
    listOfFileNames = []
    for i in range(0, len(files)):
        data = glob.glob(path + className+"/" + str(i+1) + "/*.csv")
        
        for f in data:
            listOfFileNames.append(str(i+1) + "/" + f.split("\\")[1])
    listOfDataPerClass.append(listOfFileNames)
    
listOfDataFramesPerClass = {}
dataFrameDictionary = {}
dataFrames = []

for c, d in zip(listOfClassNames, listOfDataPerClass):
    print("----------------- Word " + c + "-----------------")
    emgDataFrames = {}
    emgDataFrames['right'] = []
    emgDataFrames['left'] = []
    
    accDataFrames = {}
    accDataFrames['right'] = []
    accDataFrames['left'] = []

    gyroDataFrames = {}
    gyroDataFrames['right'] = []
    gyroDataFrames['left'] = []

    dataFrameDictionary = {}
    className = c
    emgleftctr = 0
    emgrightctr = 0
    accleftctr = 0
    accrightctr = 0
    gyroleftctr = 0
    gyrorightctr = 0
    
    for fileName in d:
        fileLocation = path + className + "/" + fileName + ""
            
        df = pd.read_csv(fileLocation)
            
        if "emg" in fileName:   
            if "-0-" in fileName:
                emgDataFrames['right'].append(df)
                emgleftctr+=1
            else:
                emgDataFrames['left'].append(df)
                emgrightctr+=1
        elif ("accelerometer") in fileName:
            if "-0-" in fileName:
                accDataFrames['right'].append(df)
                accleftctr+=1
            else:
                accDataFrames['left'].append(df)
                accrightctr+=1
        elif ("gyro") in fileName:
            if "-0-" in fileName:
                gyroDataFrames['right'].append(df)
                gyroleftctr+=1
            else:
                gyroDataFrames['left'].append(df)
                gyrorightctr+=1
    
    dataFrameDictionary["emg"] = emgDataFrames
    dataFrameDictionary["acc"] = accDataFrames
    dataFrameDictionary["gyro"] = gyroDataFrames

    listOfDataFramesPerClass[className] = dataFrameDictionary
    print("Total of " + str(emgleftctr) + " left arm emg files and " + str(emgrightctr) + " right arm emg files.")
    print("Total of " + str(accleftctr) + " left arm acc files and " + str(accrightctr) + " right arm acc files.")
    print("Total of " + str(gyroleftctr) + " left arm gyro files and " + str(gyrorightctr) + " right arm gyro files.")
    
print("DONE!")

----------------- Word binebenta-----------------
Total of 100 left arm emg files and 100 right arm emg files.
Total of 100 left arm acc files and 100 right arm acc files.
Total of 100 left arm gyro files and 100 right arm gyro files.
----------------- Word extra-----------------
Total of 100 left arm emg files and 100 right arm emg files.
Total of 100 left arm acc files and 100 right arm acc files.
Total of 100 left arm gyro files and 100 right arm gyro files.
----------------- Word gulay-----------------
Total of 100 left arm emg files and 100 right arm emg files.
Total of 100 left arm acc files and 100 right arm acc files.
Total of 100 left arm gyro files and 100 right arm gyro files.
----------------- Word gusto-----------------
Total of 100 left arm emg files and 100 right arm emg files.
Total of 100 left arm acc files and 100 right arm acc files.
Total of 100 left arm gyro files and 100 right arm gyro files.
----------------- Word itlog-----------------
Total of 100 left arm emg 

NameError: name 'end' is not defined

In [3]:
listOfTrimmedDataFramesPerClass = {}

for className in classNames:
    className = className.split("\\")[1]
    print("----------------- Word " + className + "-----------------")
    ctr = 0
    trimmedDataFramesPerClass = {}
    trimmedDataFramesPerClass['left'] = []
    trimmedDataFramesPerClass['right'] = []
    
    dataFrameDictionary = {}
    
    emgDataFrames = listOfDataFramesPerClass[className]['emg']
    accDataFrames = listOfDataFramesPerClass[className]['acc']
    gyroDataFrames = listOfDataFramesPerClass[className]['gyro']
    
    trimmedAccDataFrames = []
    trimmedGyroDataFrames = []
    trimmedEmgDataFrames = []

    
    for i in range(0,len(listOfDataFramesPerClass[className]['acc']['right'])):
        start = -1
        end = -1
        counter = 0
        print("---- %s ----" % i)
        ctr += 1
        for x in listOfDataFramesPerClass[className]['acc']['right'][i]['x']:  
            noend = False

            if(x < 0.85 and start ==-1):
                print(x)
                start = counter - 9
                if start < 0:
                    start = 0
            elif(start != -1 and x>0.90):
                end = counter + 9
                break;
                
            noend = True

            counter += 1
            
        if noend == True or end >= len(listOfDataFramesPerClass[className]['acc']['right'][i]['x']):
            print(" NO END")
            end = len(listOfDataFramesPerClass[className]['acc']['right'][i]['x'])-1
        

        startEmg = -1
        endEmg = -1
        counter2 = 0
        
        #print("Start index: %s End index: %s" % (start, end))
        for t in emgDataFrames['right'][i]['timestamp']:
            noend = False
            if(t > accDataFrames['right'][i].loc[start]['timestamp'] and startEmg == -1):
                #print("%s > %f" % (t, accDataFrames['right'][i].loc[start]['timestamp']))
                startEmg = counter2
            elif(startEmg != -1 and t > accDataFrames['right'][i].loc[end]['timestamp']):
                #print("%s > %f" % (t, accDataFrames['right'][i].loc[end]['timestamp']))
                endEmg = counter2
                break;
                
            noend = True
            counter2 += 1
        
        if noend == True:
       #     print(" NO END")
            endEmg = len(listOfDataFramesPerClass[className]['emg']['right'][i]['timestamp'])-1
        
        newAccDf = {}
        newAccDf['right'] = accDataFrames['right'][i].loc[start:end]
        newAccDf['right'] = newAccDf['right'].reset_index(drop=True)
        newAccDf['left'] = accDataFrames['left'][i].loc[start:end]
        newAccDf['left'] = newAccDf['left'].reset_index(drop=True)
        
        newGyroDf = {}
        newGyroDf['right'] = gyroDataFrames['right'][i].loc[start:end]
        newGyroDf['right'] = newGyroDf['right'].reset_index(drop=True)
        newGyroDf['left'] = gyroDataFrames['left'][i].loc[start:end]
        newGyroDf['left'] = newGyroDf['left'].reset_index(drop=True)
        
        newEmgDf = {}
        newEmgDf['right'] = emgDataFrames['right'][i].loc[startEmg:endEmg]
        newEmgDf['right'] = newEmgDf['right'].reset_index(drop=True)
        newEmgDf['left'] = emgDataFrames['left'][i].loc[startEmg:endEmg]
        newEmgDf['left'] = newEmgDf['left'].reset_index(drop=True)

        trimmedAccDataFrames.append(newAccDf)
        trimmedGyroDataFrames.append(newGyroDf)
        trimmedEmgDataFrames.append(newEmgDf)
        
        #print("Start emg index: %s End emg index: %s" % (startEmg, endEmg))
        
    dataFrameDictionary['acc'] = trimmedAccDataFrames
    dataFrameDictionary['gyro'] = trimmedGyroDataFrames
    dataFrameDictionary['emg'] = trimmedEmgDataFrames
        
    listOfTrimmedDataFramesPerClass[className] = dataFrameDictionary

        
    
    print("Done Trimming " + className + " " + str(ctr) + " files")


----------------- Word binebenta-----------------
---- 0 ----
0.692383
---- 1 ----
0.837402
---- 2 ----
0.827637
---- 3 ----
0.811035
---- 4 ----
0.834473
---- 5 ----
0.831543
---- 6 ----
0.824707
---- 7 ----
0.818848
---- 8 ----
0.832031
---- 9 ----
0.798828
---- 10 ----
0.78125
---- 11 ----
0.811523
---- 12 ----
0.813477
---- 13 ----
0.806641
---- 14 ----
0.8125
---- 15 ----
0.848145
---- 16 ----
0.836914
---- 17 ----
0.82959
---- 18 ----
0.819336
---- 19 ----
0.848633
---- 20 ----
0.720215
---- 21 ----
0.787598
---- 22 ----
0.782227
---- 23 ----
0.678223
---- 24 ----
0.837402
---- 25 ----
0.837402
---- 26 ----
0.771484
---- 27 ----
0.805664
---- 28 ----
0.807129
---- 29 ----
0.674805
---- 30 ----
0.748535
---- 31 ----
0.747559
---- 32 ----
0.730469
---- 33 ----
0.791016
---- 34 ----
0.832031
---- 35 ----
-1.01367
---- 36 ----
0.831543
---- 37 ----
0.743164
---- 38 ----
0.800293
---- 39 ----
0.758301
---- 40 ----
0.79541
---- 41 ----
0.756348
---- 42 ----
0.82666
---- 43 ----
0.74658

---- 62 ----
0.82373
---- 63 ----
0.846191
---- 64 ----
0.799316
---- 65 ----
0.84375
---- 66 ----
0.780273
---- 67 ----
0.803223
---- 68 ----
0.831543
---- 69 ----
0.844727
---- 70 ----
0.830566
---- 71 ----
0.827148
---- 72 ----
0.840332
 NO END
---- 73 ----
0.84375
---- 74 ----
0.835449
---- 75 ----
0.645996
---- 76 ----
0.681152
---- 77 ----
0.758301
---- 78 ----
0.786621
---- 79 ----
0.766602
---- 80 ----
0.791504
---- 81 ----
0.744629
---- 82 ----
0.79834
---- 83 ----
0.847168
---- 84 ----
0.82373
---- 85 ----
0.80127
---- 86 ----
0.801758
---- 87 ----
0.778809
---- 88 ----
0.847168
---- 89 ----
0.838867
---- 90 ----
0.73584
---- 91 ----
0.833008
---- 92 ----
0.831543
---- 93 ----
0.837891
---- 94 ----
0.792969
---- 95 ----
0.799805
---- 96 ----
0.818359
---- 97 ----
0.784668
---- 98 ----
0.825684
---- 99 ----
0.822754
Done Trimming gusto 100 files
----------------- Word itlog-----------------
---- 0 ----
0.835938
---- 1 ----
0.819824
---- 2 ----
0.842285
---- 3 ----
0.842773
---

---- 20 ----
0.827148
---- 21 ----
0.848633
---- 22 ----
0.841797
---- 23 ----
0.768066
---- 24 ----
0.82666
---- 25 ----
0.844727
---- 26 ----
0.838379
---- 27 ----
0.804199
---- 28 ----
0.822266
---- 29 ----
0.802246
---- 30 ----
0.710449
---- 31 ----
0.80957
---- 32 ----
0.822266
---- 33 ----
0.716309
---- 34 ----
0.833008
---- 35 ----
0.838867
---- 36 ----
0.789551
---- 37 ----
0.747559
---- 38 ----
0.687988
---- 39 ----
0.84375
---- 40 ----
0.837402
---- 41 ----
0.828125
---- 42 ----
0.849609
---- 43 ----
0.845215
---- 44 ----
0.450684
---- 45 ----
0.763184
---- 46 ----
0.847168
---- 47 ----
0.844727
---- 48 ----
0.821777
---- 49 ----
0.837891
---- 50 ----
0.778809
---- 51 ----
0.812012
---- 52 ----
0.80127
---- 53 ----
0.818848
---- 54 ----
0.834473
---- 55 ----
0.784668
---- 56 ----
0.843262
---- 57 ----
0.800293
---- 58 ----
0.725098
---- 59 ----
0.808594
---- 60 ----
0.826172
---- 61 ----
0.843262
---- 62 ----
0.795898
---- 63 ----
0.840332
---- 64 ----
0.847168
---- 65 ----
0

---- 84 ----
0.781738
---- 85 ----
0.744629
---- 86 ----
0.800781
---- 87 ----
0.814941
---- 88 ----
0.785645
---- 89 ----
0.800781
---- 90 ----
0.735352
---- 91 ----
0.798828
---- 92 ----
0.82666
---- 93 ----
0.827637
---- 94 ----
0.805664
---- 95 ----
0.719727
---- 96 ----
0.749023
---- 97 ----
0.830078
---- 98 ----
0.79248
---- 99 ----
0.765625
Done Trimming maliit 100 files
----------------- Word mayroon-----------------
---- 0 ----
0.752441
---- 1 ----
0.846191
---- 2 ----
0.839355
---- 3 ----
0.797363
---- 4 ----
0.768555
---- 5 ----
0.846191
---- 6 ----
0.811523
---- 7 ----
0.831543
---- 8 ----
0.821777
---- 9 ----
0.772949
---- 10 ----
0.644531
---- 11 ----
0.785156
---- 12 ----
0.80957
---- 13 ----
0.731445
---- 14 ----
0.724121
---- 15 ----
0.849121
---- 16 ----
0.67041
---- 17 ----
0.836426
---- 18 ----
0.813965
---- 19 ----
0.843262
---- 20 ----
0.750977
---- 21 ----
0.6875
---- 22 ----
0.444824
---- 23 ----
0.754395
---- 24 ----
0.837402
---- 25 ----
0.728027
---- 26 ----


---- 44 ----
0.729492
---- 45 ----
0.791504
---- 46 ----
0.849121
---- 47 ----
0.774902
---- 48 ----
0.75293
---- 49 ----
0.695313
---- 50 ----
0.791504
---- 51 ----
0.814941
---- 52 ----
0.816406
---- 53 ----
0.761719
---- 54 ----
0.772461
---- 55 ----
0.724609
---- 56 ----
0.740723
---- 57 ----
0.794434
---- 58 ----
0.838867
---- 59 ----
0.783691
---- 60 ----
0.849121
---- 61 ----
0.835938
---- 62 ----
0.831543
---- 63 ----
0.836426
---- 64 ----
0.837402
---- 65 ----
0.830078
---- 66 ----
0.776367
---- 67 ----
0.819824
---- 68 ----
0.812012
---- 69 ----
0.740723
 NO END
---- 70 ----
0.764648
---- 71 ----
0.804199
---- 72 ----
0.786621
---- 73 ----
0.713867
---- 74 ----
0.685547
---- 75 ----
0.708496
---- 76 ----
0.786621
---- 77 ----
0.825684
---- 78 ----
0.73291
---- 79 ----
0.848633
---- 80 ----
0.680176
---- 81 ----
0.797852
---- 82 ----
0.821777
---- 83 ----
0.775879
---- 84 ----
0.737793
---- 85 ----
0.792969
---- 86 ----
0.836914
---- 87 ----
0.849609
---- 88 ----
0.819824
----

---- 5 ----
0.836914
---- 6 ----
0.774414
---- 7 ----
0.800293
 NO END
---- 8 ----
0.847168
 NO END
---- 9 ----
0.831543
---- 10 ----
0.799805
---- 11 ----
0.804199
---- 12 ----
0.713867
---- 13 ----
0.810547
---- 14 ----
0.703613
---- 15 ----
0.818848
---- 16 ----
0.839355
---- 17 ----
0.813477
---- 18 ----
0.849121
---- 19 ----
0.841797
---- 20 ----
0.834961
---- 21 ----
0.775879
---- 22 ----
0.82959
---- 23 ----
0.786621
---- 24 ----
0.815918
---- 25 ----
0.778809
---- 26 ----
0.841797
---- 27 ----
0.842285
---- 28 ----
0.828613
---- 29 ----
0.636719
---- 30 ----
0.769043
---- 31 ----
0.804688
---- 32 ----
0.739258
---- 33 ----
0.848633
---- 34 ----
0.835449
---- 35 ----
0.80127
---- 36 ----
0.831543
---- 37 ----
0.814941
---- 38 ----
0.830566
---- 39 ----
0.697266
---- 40 ----
0.841309
---- 41 ----
0.780762
---- 42 ----
0.839844
---- 43 ----
0.824219
---- 44 ----
0.845215
---- 45 ----
0.599609
---- 46 ----
0.771973
---- 47 ----
0.628906
---- 48 ----
0.685059
---- 49 ----
0.814453
-

---- 68 ----
0.802734
---- 69 ----
0.838379
---- 70 ----
0.786621
---- 71 ----
0.724121
---- 72 ----
0.828125
---- 73 ----
0.820801
---- 74 ----
0.821777
---- 75 ----
0.772461
---- 76 ----
0.799316
---- 77 ----
0.766113
---- 78 ----
0.818848
---- 79 ----
0.801758
---- 80 ----
0.766113
---- 81 ----
0.830078
---- 82 ----
0.826172
---- 83 ----
0.752441
---- 84 ----
0.845215
---- 85 ----
0.813477
---- 86 ----
0.787598
---- 87 ----
0.807129
---- 88 ----
0.758301
---- 89 ----
0.820313
---- 90 ----
0.719727
---- 91 ----
0.798828
---- 92 ----
0.812012
---- 93 ----
0.835449
---- 94 ----
0.818848
---- 95 ----
0.705078
---- 96 ----
0.765137
---- 97 ----
0.748535
---- 98 ----
0.845215
---- 99 ----
0.754883
Done Trimming size 100 files
----------------- Word spaghetti-----------------
---- 0 ----
0.796387
---- 1 ----
0.818848
---- 2 ----
0.363281
---- 3 ----
0.839844
---- 4 ----
0.681152
---- 5 ----
0.697266
---- 6 ----
0.824219
---- 7 ----
0.831055
---- 8 ----
0.783691
---- 9 ----
0.830078
---- 10

---- 30 ----
0.794434
---- 31 ----
0.815918
---- 32 ----
0.818359
---- 33 ----
0.722656
---- 34 ----
0.821777
---- 35 ----
0.782715
---- 36 ----
0.837402
---- 37 ----
0.769043
---- 38 ----
0.819336
---- 39 ----
0.84668
---- 40 ----
0.833008
---- 41 ----
0.804199
---- 42 ----
0.798828
---- 43 ----
0.776855
---- 44 ----
0.776367
---- 45 ----
0.840332
---- 46 ----
0.770508
---- 47 ----
0.84082
---- 48 ----
0.750488
---- 49 ----
0.836426
---- 50 ----
0.837402
---- 51 ----
0.832031
---- 52 ----
0.847656
---- 53 ----
0.808594
---- 54 ----
0.779297
---- 55 ----
0.48877
---- 56 ----
0.762695
---- 57 ----
0.836426
---- 58 ----
0.845215
---- 59 ----
0.785645
---- 60 ----
0.785156
---- 61 ----
0.845215
---- 62 ----
0.84668
---- 63 ----
0.844238
---- 64 ----
0.836914
---- 65 ----
0.841309
---- 66 ----
0.847656
---- 67 ----
0.827637
---- 68 ----
0.817871
---- 69 ----
0.818848
---- 70 ----
0.786133
---- 71 ----
0.802246
---- 72 ----
0.809082
---- 73 ----
0.833008
---- 74 ----
0.79248
---- 75 ----
0.

---- 94 ----
0.847656
---- 95 ----
0.808105
---- 96 ----
0.804688
---- 97 ----
0.835449
---- 98 ----
0.807617
---- 99 ----
0.839844
Done Trimming wala 100 files


In [4]:
import os

directory = "trimmed_data(2arms)/"
if not os.path.exists(directory):
    os.makedirs(directory)
    
for className in classNames:
    className = className.split("\\")[1]
    
    if not os.path.exists(directory +className):
        os.makedirs(directory  +className)
    
    ctr = 0
    for df in listOfTrimmedDataFramesPerClass[className]['emg']:
        fileName = directory + className + "/emg" + str(ctr)
        df['right'].to_csv(fileName + "-Right2a.csv", encoding='utf-8', index=False)
        df['left'].to_csv(fileName + "-Left2a.csv", encoding='utf-8', index=False)
        ctr+=1
    
    ctr = 0
    for df in listOfTrimmedDataFramesPerClass[className]['acc']:
        fileName = directory + className + "/acc" + str(ctr)
        df['right'].to_csv(fileName + "-Right2a.csv", encoding='utf-8', index=False)
        df['left'].to_csv(fileName + "-Left2a.csv", encoding='utf-8', index=False)
        ctr+=1

    ctr = 0
    for df in listOfTrimmedDataFramesPerClass[className]['gyro']:
        fileName = directory + className + "/gyro" + str(ctr)
        df['right'].to_csv(fileName + "-Right2a.csv", encoding='utf-8', index=False)
        df['left'].to_csv(fileName + "-Left2a.csv", encoding='utf-8', index=False)
        ctr+=1
    
    print("Done saving trimmed files of " + className)

print("DONE")

Done saving trimmed files of binebenta
Done saving trimmed files of extra
Done saving trimmed files of gulay
Done saving trimmed files of gusto
Done saving trimmed files of itlog
Done saving trimmed files of juice
Done saving trimmed files of kape
Done saving trimmed files of kilo
Done saving trimmed files of mahal
Done saving trimmed files of malaki
Done saving trimmed files of maliit
Done saving trimmed files of mayroon
Done saving trimmed files of nagbubukas
Done saving trimmed files of nagsasara
Done saving trimmed files of oras
Done saving trimmed files of pabili
Done saving trimmed files of poloshirt
Done saving trimmed files of puwede
Done saving trimmed files of saging
Done saving trimmed files of sari-sari store
Done saving trimmed files of shorts
Done saving trimmed files of size
Done saving trimmed files of spaghetti
Done saving trimmed files of supot
Done saving trimmed files of t-shirt
Done saving trimmed files of tawad
Done saving trimmed files of tinapay
Done saving trim

In [5]:
listOfTrimmedDataFramesPerClass[className]['emg'][0]['right']

,timestamp,emg1,emg2,emg3,emg4,emg5,emg6,emg7,emg8
0,1528874823151197,-1,-1,-2,-3,13,5,-2,0
1,1528874823151197,-2,-2,-4,1,-8,10,1,-2
2,1528874823158178,-1,0,6,3,-17,-18,-4,-3
3,1528874823158178,-2,0,-1,-2,18,-7,-2,1
4,1528874823173139,-3,-4,-3,13,9,15,-1,-3
5,1528874823173139,0,-3,-12,-19,-4,0,2,0
6,1528874823182115,-1,2,10,8,11,12,3,1
7,1528874823182115,0,2,9,19,2,18,1,0
8,1528874823188098,-1,-3,-12,-22,-56,-58,-11,-4
9,1528874823188098,-4,-5,2,8,-9,-40,-15,-6


In [6]:
# trimmedAccDataFrames = {}
# trimmedAccDataFrames['right'] = []
# trimmedAccDataFrames['left'] = []

# trimmedGyroDataFrames = {}
# trimmedGyroDataFrames['right'] = []
# trimmedGyroDataFrames['left'] = []

# trimmedEmgDataFrames = {}
# trimmedEmgDataFrames['right'] = []
# trimmedEmgDataFrames['left'] = []

# for i in range(0,len(gyroDataFrames['right'])):
#     start = -1
#     end = -1
#     counter = 0
#     for x in accDataFrames['right'][i]['x']:  
#         if(x < 0.85 and start ==-1):
#             print(x)
#             start = counter - 10
#         elif(start != -1 and x>0.99):
#             end = counter + 9
#             break;
#         counter += 1
    
#     startEmg = -1
#     endEmg = -1
#     counter2 = 0
#     for t in emgDataFrames['right'][i]['timestamp']:
#         if(t > accDataFrames['right'][i].loc[start]['timestamp'] and startEmg == -1):
#             #print("%s > %f" % (t, accDataFrames['right'][i].loc[start]['timestamp']))
#             startEmg = counter2-1
#         elif(startEmg != -1 and t > accDataFrames['right'][i].loc[end]['timestamp']):
#             #print("%s > %f" % (t, accDataFrames['right'][i].loc[end]['timestamp']))
#             endEmg = counter2-1
#             break;
#         counter2 += 1
    
#     newAccDf = accDataFrames['right'][i].loc[start:end]
#     newAccDf = newAccDf.reset_index(drop=True)
#     newGyroDf = gyroDataFrames['right'][i].loc[start:end]
#     newGyroDf = newGyroDf.reset_index(drop=True)
#     newEmgDf = emgDataFrames['right'][i].loc[startEmg:endEmg]
#     newEmgDf = newEmgDf.reset_index(drop=True)

#     trimmedAccDataFrames['right'].append(newAccDf)
#     trimmedGyroDataFrames['right'].append(newGyroDf)
#     trimmedEmgDataFrames['right'].append(newEmgDf)
    
#     print("Start index: %s End index: %s" % (start, end))
#     print("Start emg index: %s End emg index: %s" % (startEmg, endEmg))


In [7]:
# import glob
# import pandas as pd

# listOfFileNames = []
# listOfClassNames = []
# listOfDataPerClass = []
# files = []
# classNames = []

# files = glob.glob("/Users/user/Desktop/FSL by non-signers/ano/1/*.csv")
# print(len(files))
# listOfFileNames = []
# for f in files:
#     listOfFileNames.append(f.split("\\")[1])
    
# emgDataFrames = {}
# emgDataFrames['right'] = []
# emgDataFrames['left'] = []
    
# accDataFrames = {}
# accDataFrames['right'] = []
# accDataFrames['left'] = []

# gyroDataFrames = {}
# gyroDataFrames['right'] = []
# gyroDataFrames['left'] = []

# dataFrameDictionary = {}
# leftctr = 0
# rightctr = 0
# for fileName in listOfFileNames:
#     fileLocation = "/Users/user/Desktop/FSL by non-signers/ano/1/"+fileName 
#     #print(fileLocation)
#     df = pd.read_csv(fileLocation)
    
#     if "emg" in fileName:   
#         if "-0-" in fileName:
#             emgDataFrames['right'].append(df)
#             leftctr+=1
#         else:
#             emgDataFrames['left'].append(df)
#             rightctr+=1
#     elif ("accelerometer") in fileName:
#         if "-0-" in fileName:
#             accDataFrames['right'].append(df)
#             leftctr+=1
#         else:
#             accDataFrames['left'].append(df)
#             rightctr+=1
#     elif ("gyro") in fileName:
#         if "-0-" in fileName:
#             gyroDataFrames['right'].append(df)
#             leftctr+=1
#         else:
#             gyroDataFrames['left'].append(df)      
#             rightctr+=1

# print("Total of " + str(leftctr) + " left arm files and " + str(rightctr) + " right arm files.")

    
# #listOfClassNames
# #listOfDataPerClass[0]
# print("Done listing the class and file names")

# Pancheck if goods yung trim

In [ ]:
# import matplotlib.pyplot as plt

# for i in range(0,len(trimmedAccDataFrames['right'])):
#     print("-----%s------" % str(i+1))
#     print("Untrimmed")

#     fig_size = plt.rcParams["figure.figsize"]    
#     fig_size[0] = 14
#     fig_size[1] = 4
#     plt.rcParams["figure.figsize"] = fig_size

#     plt.plot(accDataFrames['right'][i]['x'])
#     plt.plot(accDataFrames['right'][i]['y'])
#     plt.plot(accDataFrames['right'][i]['z'])
    
# #     plt.plot(emgDataFrames['right'][i]['emg1'])
# #     plt.plot(emgDataFrames['right'][i]['emg2'])
# #     plt.plot(emgDataFrames['right'][i]['emg3'])
# #     plt.plot(emgDataFrames['right'][i]['emg4'])
# #     plt.plot(emgDataFrames['right'][i]['emg5'])
# #     plt.plot(emgDataFrames['right'][i]['emg6'])
# #     plt.plot(emgDataFrames['right'][i]['emg7'])
# #     plt.plot(emgDataFrames['right'][i]['emg8'])
    
# #     plt.plot(gyroDataFrames['right'][i]['x'])
# #     plt.plot(gyroDataFrames['right'][i]['y'])
# #     plt.plot(gyroDataFrames['right'][i]['z'])
#     plt.show()
#     print("Trimmed")

#     fig_size = plt.rcParams["figure.figsize"]    
#     fig_size[0] = 14
#     fig_size[1] = 4
#     plt.rcParams["figure.figsize"] = fig_size
#     plt.plot(trimmedAccDataFrames['right'][i]['x'])
#     plt.plot(trimmedAccDataFrames['right'][i]['y'])
#     plt.plot(trimmedAccDataFrames['right'][i]['z'])
    
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg1'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg2'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg3'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg4'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg5'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg6'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg7'])
# #     plt.plot(trimmedEmgDataFrames['right'][i]['emg8'])

# #     plt.plot(trimmedGyroDataFrames['right'][i]['x'])
# #     plt.plot(trimmedGyroDataFrames['right'][i]['y'])
# #     plt.plot(trimmedGyroDataFrames['right'][i]['z'])
#     plt.show()

In [ ]:
# import numpy as np

# start = -1
# end = -1
# counter = 0
# for x in accDataFrames['right'][1]['x']:  
#     mean = np.mean(accDataFrames['right'][0]['x'].loc[0:counter])
#     sd = np.std(accDataFrames['right'][0]['x'].loc[0:counter])*4
#     print("n: %s mean: %s sd: %s mean+sd = %s" % (accDataFrames['right'][0]['x'][counter], mean, sd, mean+sd))
    
#     if(counter>0):
#         if(accDataFrames['right'][1]['y'][counter+1] < (mean-sd)):
#             print(counter+1)
#             break;
#     counter +=1
    


# Save file